In [23]:
from normalization.components import Relvar, FunctionalDependency, MultivaluedDependency, Attribute
import numpy as np
import matplotlib.pyplot as plt

In [24]:
df1 = FunctionalDependency("{CLABE} -> {Nombre, Apellido, RFC}")
df1.is_trivial()

False

In [25]:
dmv1 = MultivaluedDependency("{RFC} ->-> {CLABE}")
dmv1.is_trivial(heading={Attribute("CLABE"), Attribute("RFC")})

True

In [26]:
df1 = FunctionalDependency("{A} -> {B}")
df2 = FunctionalDependency("{B} -> {C}")
df3 = FunctionalDependency("{A,D} -> {E}")

def closure(attributes: set[Attribute], functional_dependencies: set[FunctionalDependency]) -> set[Attribute]:
    finished = False

    while not finished:
        initial = len(attributes)
        for fd in functional_dependencies:
            if fd.determinant.issubset(attributes):
                attributes = attributes.union(fd.dependant)
        if initial == len(attributes):
            finished = True

    return attributes

closure(
    attributes={Attribute("A")},
    functional_dependencies={df1,df2,df3}
)

{Attribute(name='A'), Attribute(name='B'), Attribute(name='C')}

In [27]:
df1 = FunctionalDependency("{A} -> {B, E}")
df2 = FunctionalDependency("{B} -> {C}")
df3 = FunctionalDependency("{A,D} -> {E}")
df4 = FunctionalDependency("{E} -> {A,B,C}")

def is_superkey(attributes: set[Attribute], heading: set[Attribute], functional_dependencies: set[FunctionalDependency]) -> bool:
    closure_set = closure(
        attributes=attributes,
        functional_dependencies=functional_dependencies
    )
    return heading.issubset(closure_set)

is_superkey(
    attributes={Attribute("A"), Attribute("E")},
    functional_dependencies={df1,df2,df3},
    heading={Attribute(name='A'), Attribute(name='B'), Attribute(name='C'),  Attribute(name='E')}
)

True

In [28]:
def is_key(attributes: set[Attribute], heading: set[Attribute], functional_dependencies: set[FunctionalDependency]) -> bool:
    is_spk = is_superkey(
        attributes=attributes,
        heading=heading,
        functional_dependencies=functional_dependencies
    )
    if not is_spk:
        return False

    for attr in attributes:
        aux = attributes - {attr}
        if is_superkey(aux, heading, functional_dependencies):
            return False

    return True


In [29]:
df1 = FunctionalDependency("{A} -> {B,D}")
df2 = FunctionalDependency("{B} -> {C}")

is_key(
    attributes={Attribute("A")},
    functional_dependencies={df1,df2},
    heading={Attribute(name='A'), Attribute(name='B'), Attribute(name='C'), Attribute(name='D')}
)

True

In [30]:
def is_relvar_in_bcnf(relvar: Relvar):
    non_trivial_fd = {fd for fd in relvar.functional_dependencies if not fd.is_trivial()}

    for fd in non_trivial_fd:
        if not is_superkey(
            fd.determinant,
            relvar.heading,
            relvar.functional_dependencies
        ):
            return False

    return True

In [31]:
fd1 = FunctionalDependency("{A} -> {B}")
fd2 = FunctionalDependency("{B} -> {C}")
fd3 = FunctionalDependency("{C} -> {A}")
relvar = Relvar(
    {"A", "B", "C"},
    {fd1, fd2, fd3}
)

is_relvar_in_bcnf(relvar)


True

In [32]:
def is_relvar_in_4nf(relvar: Relvar):
    non_trivial_mfd = {mfd for mfd in relvar.multivalued_dependencies if not mfd.is_trivial(relvar.heading)}

    for mfd in non_trivial_mfd:
        if not is_superkey(
            mfd.determinant,
            relvar.heading,
            relvar.functional_dependencies
        ):
            return False

    return True

In [ ]:
fd = FunctionalDependency("{EstudianteID} -> {Curso, Club}")

mfd1 = MultivaluedDependency("{EstudianteID} ->-> {Curso}")
mfd2 = MultivaluedDependency("{EstudianteID} ->-> {Club}")

relvar_4nf = Relvar(
    heading={"EstudianteID", "Curso", "Club"},
    functional_dependencies={fd},
    multivalued_dependencies={mfd1, mfd2}
)

print(is_relvar_in_4nf(relvar_4nf))  # Debería imprimir True

True


In [ ]:
fd = FunctionalDependency("{A} -> {B}")

mfd = MultivaluedDependency("{B} ->-> {C}")

relvar_not_4nf = Relvar(
    {"A", "B", "C"},
    functional_dependencies={fd},
    multivalued_dependencies={mfd}
)

print(is_relvar_in_4nf(relvar_not_4nf))  # Debería imprimir False

False


In [36]:
mvd1 = MultivaluedDependency("{EmpleadoID} ->-> {Proyecto}")

mvd2 = MultivaluedDependency("{EmpleadoID} ->-> {Habilidad}")

relvar_only_mvd = Relvar(
    {"EmpleadoID", "Proyecto", "Habilidad"},
    multivalued_dependencies={mvd1, mvd2}
)

print(is_relvar_in_4nf(relvar_only_mvd))  # Debería imprimir False

False
